In [19]:
import pandas as pd
import os

In [20]:
from pathlib import Path
from germanetpy.germanet import Germanet
from germanetpy.frames import Frames
from germanetpy.filterconfig import Filterconfig
from germanetpy.synset import WordCategory, WordClass

In [21]:
path2 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1.csv"
Emo = pd.read_csv(path2, sep=",")

Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_M_SD,EM_O_F_SD,EM_Y_N,EM_O_N,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,27.38,51.56,18,19,5.83,36.16,36.58,41.77,0.0,0.0
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,34.06,21.04,19,19,-49.37,-62.74,24.86,32.97,0.0,0.0
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,26.29,25.94,18,19,42.83,50.47,27.44,26.39,0.0,0.0
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,22.06,36.59,19,20,28.79,59.00,23.44,29.92,0.0,0.0
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,34.31,25.50,18,19,-39.78,-45.63,23.08,28.66,0.0,0.0


In [22]:
germanet_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/Germanet/GN_V180/GN_V180_XML/"

#frequencylist_nouns = data_path + "/noun_freqs_decow14_16.txt"
germanet = Germanet(germanet_path)

Load GermaNet data...: 100%|█████████▉| 99.99999999999996/100 [00:05<00:00, 19.99it/s] 
Load Wictionary data...: 100%|██████████| 100.0/100 [00:00<00:00, 2011.39it/s]
Load Ili records...: 100%|██████████| 100.0/100 [00:00<00:00, 372165.39it/s]


## Idea: Categorise words based on nature by identifying if the words share a similar hypernym-node 

In [ ]:
#[Synset(id=s39164, lexunits=Aal),
 #Synset(id=s49868, lexunits=Aal, Aalfisch, Flussaal)]

### Inspect synsets based on words and synset id 

In [49]:
test_cases = ['Adler', 'Admiral', 'Affe', 'Alge', 'Amateur', 'Angeber']

word = 'Adler'
id = 's106273'

word_synsets = germanet.get_synsets_by_orthform(word)#[0]
synset_id = germanet.get_synset_by_id(id)

#Hypernyms = word_synsets.all_hypernyms()
Hypernyms = synset_id.all_hypernyms()

#synset_id
#Hypernyms
word_synsets

#Synset(id=s34063, lexunits=Mensch, Person, Persönlichkeit, Individuum, Menschenwesen)

[Synset(id=s50142, lexunits=Adler, Aar)]

In [50]:

# loop for identifying if a single word (synset_id) is nature based human, nature based non-human or neither 
def find_synset_by_id(id):
    try:
        word_synsets = germanet.get_synset_by_id(id)
        hypernyms = word_synsets.all_hypernyms()
    except:
        return False
    
    has_s34063 = False
    has_s50990 = False
    
    for synset in hypernyms:
        if synset.id == "s34063":
            has_s34063 = True
        if synset.id == "s50990":
            has_s50990 = True
        
    if has_s34063 == True and has_s50990 == True:
        return "nature_based_human"
    elif has_s34063 == False and has_s50990 == True:
        return "nature_based_non_human"
    else:
        return ""

find_synset_by_id("s50142")


'nature_based_non_human'

In [77]:
word = "Admiral"
synsets = germanet.get_synsets_by_orthform(word)
print(synsets)

#hypernyms_test = synsets[1].all_hypernyms()
#hypernyms_test

#find_synset_by_id(synsets.id)
find_synset_by_id(synsets[1].id)

[Synset(id=s49365, lexunits=Admiral), Synset(id=s36988, lexunits=Admiral, Admiralin)]


'nature_based_human'

In [79]:
# loop through each synset of a given word to identify if one of the synsets is nature based 
def categorise_synset(word):
    synsets = germanet.get_synsets_by_orthform(word)
    has_human_meaning = False
    has_non_human_meaning = False
    
    for synset in synsets:
        result = find_synset_by_id(synset.id)
        if result == 'nature_based_human':
            has_human_meaning = True
        if result == 'nature_based_non_human':
            has_non_human_meaning = True
            
    if has_human_meaning == True and has_non_human_meaning == True:
        return "nature_based_non_human"
    elif has_human_meaning == False and has_non_human_meaning == True:
        return "nature_based_non_human"
    elif has_human_meaning == True and has_non_human_meaning == False:
        return "nature_based_human"
    else:
        return ""
    
        
print(categorise_synset("Affe"))

nature_based_non_human


In [81]:
# testcases and automated testing
test_cases = ['Ratte', 'Schlange', 'Aal', 'Adler', 'Admiral', 'Affe', 'Alge', 'Amateur', 'Angeber', 'Tisch', 'Kaffee']

for test in test_cases:
    print(f"Result for {test}: {categorise_synset(test)}")

Result for Ratte: nature_based_non_human
Result for Schlange: nature_based_non_human
Result for Aal: nature_based_non_human
Result for Adler: nature_based_non_human
Result for Admiral: nature_based_non_human
Result for Affe: nature_based_non_human
Result for Alge: nature_based_non_human
Result for Amateur: nature_based_human
Result for Angeber: nature_based_human
Result for Tisch: 
Result for Kaffee: 


In [82]:
# create a new column in the Emo df and categorise words 
Emo["Kategorie"] = Emo["WORD"].apply(categorise_synset)
Emo[["WORD", "Kategorie"]].head(50)

,WORD,Kategorie
0,Aal,nature_based_non_human
1,Aas,
2,Abend,
3,Abendbrot,
4,Abfall,
5,Abgrund,
6,Abhang,
7,Abnahme,
8,Abrechnung,
9,Abruf,


In [83]:
counts = Emo["Kategorie"].value_counts()

print(counts)

Kategorie
                          1969
nature_based_non_human     341
nature_based_human         282
Name: count, dtype: int64


In [91]:
#Inspect result in excel
path = '/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/'
file_xlsx = os.path.join(path, "Emo_categorised.xlsx")
file_csv = os.path.join(path, "Emo_categorised.csv")

#Emo.to_excel(file, index=False)
Emo.to_csv(file_csv, index=False)

In [85]:
os.getcwd()

'/Users/Richard/PycharmProjects/NeuroLab_3.10/Germanet/Code'

In [200]:
out_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1_categorised.csv"

Emo.to_csv(out_path, index=False)

In [199]:
Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_F_SD,EM_Y_N,EM_O_N,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,51.56,18,19,5.83,36.16,36.58,41.77,0.0,0.0,nature_based_non_human
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,21.04,19,19,-49.37,-62.74,24.86,32.97,0.0,0.0,None
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,25.94,18,19,42.83,50.47,27.44,26.39,0.0,0.0,None
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,36.59,19,20,28.79,59.00,23.44,29.92,0.0,0.0,None
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,25.50,18,19,-39.78,-45.63,23.08,28.66,0.0,0.0,None


In [198]:
Emo.drop(columns=['Path_distance', 'Path_distance_string'], inplace=True)